In [1]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [2]:
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = MNIST('.', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [3]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [4]:
model = autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/100], loss:0.1963
epoch [2/100], loss:0.1646
epoch [3/100], loss:0.1591
epoch [4/100], loss:0.1591
epoch [5/100], loss:0.1566
epoch [6/100], loss:0.1552
epoch [7/100], loss:0.1427
epoch [8/100], loss:0.1483
epoch [9/100], loss:0.1525
epoch [10/100], loss:0.1386
epoch [11/100], loss:0.1524
epoch [12/100], loss:0.1384
epoch [13/100], loss:0.1366
epoch [14/100], loss:0.1432
epoch [15/100], loss:0.1372
epoch [16/100], loss:0.1331
epoch [17/100], loss:0.1384
epoch [18/100], loss:0.1337
epoch [19/100], loss:0.1334
epoch [20/100], loss:0.1377
epoch [21/100], loss:0.1381
epoch [22/100], loss:0.1337
epoch [23/100], loss:0.1315
epoch [24/100], loss:0.1315
epoch [25/100], loss:0.1311
epoch [26/100], loss:0.1229
epoch [27/100], loss:0.1361
epoch [28/100], loss:0.1241
epoch [29/100], loss:0.1284
epoch [30/100], loss:0.1286
epoch [31/100], loss:0.1289
epoch [32/100], loss:0.1301
epoch [33/100], loss:0.1259
epoch [34/100], loss:0.1258
epoch [35/100], loss:0.1227
epoch [36/100], loss:0.1264
e